# COMP1801 Tutorial Week 4 - More Complicated Models, Overfitting and Regularisation

*Dr Peter Soar - 2024/25*

This week we will be expanding on the concept of overfitting we were introduced to last week, exploring how we can mitigate this with data splitting, hyperparameter selection, regularisation and finally revising the general model selection process we should be following when performing supervised learning. But first we will start with a quick look at how to deal with categorical features in our data.

For this tutorial, read through and try to understand the text and code examples I have provided (ask your tutor if you have any questions) and there will be a selection of exercises. Attempt these exercises on your own, but do ask your tutor for help if you get stuck.

##0. Do not forget to import all the Python Libraries being used!

In [ ]:
import numpy as np # A useful package for dealing with mathematical processes, we will be using it this week for vectors and matrices
import pandas as pd # A common package for viewing tabular data
import sklearn.linear_model, sklearn.datasets # We want to be able to access the sklearn datasets again, also we are using some model evaluation
from sklearn.preprocessing import StandardScaler, MinMaxScaler, PolynomialFeatures, LabelEncoder, OneHotEncoder # We will be using the inbuilt preprocessing functions sklearn provides
import matplotlib.pyplot as plt # We will be using Matplotlib for our graphs
from sklearn.model_selection import train_test_split # A library that can automatically perform data splitting for us
from sklearn.linear_model import Ridge, Lasso, LinearRegression # Ridge & Lasso regression are types of linear model that use regularisation
from sklearn.metrics import mean_squared_error, r2_score # Allows us to use the MSE function without calling in sklearn each time
from google.colab import files # We will be importing a csv file I have provided for one section.
import warnings
warnings.filterwarnings('ignore') # suppresses a convergence warning we may get when testing Lasso - this is just cosmetic to stop too much output.

# 1. Categorical feature data

So far we have only been considering feature columns that have been populated with numerical data, however it is not unusual to encounter datasets where the feature columns may include text values that have some meaning for the target predictions, however these must be converted into a numerical form somehow in order to be used by the machine learning algorithms.

We would call these categorical feature data, and converting these features into numerical values (often called 'feature encoding') is another data preprocessing operation you should be aware of when using tabular data.

##1.1 Loading in dataset

For this problem, I have created a synthetic dataset which I have uploaded to moodle. Please load this csv below.



In [ ]:
uploaded = files.upload()    # Will prompt you to select file
# Your file is now in the Colab filesystem on the left
testData = pd.read_csv('testData.csv') # Save it to a pandas dataframe

This data shows the final grade of a selection of students from 4 different schools. We have the scored from their previous two mock papers, but there are also a number of categorical feature columns that we can't currently use.

Let's have a look at the data:

In [ ]:
# This file has all the data elements constructed already, so we can just put the entire thing into the 'data='
df = pd.DataFrame(data= testData)

# show the data
display(df)

First, let's see how good a fit we get just using the mock scores, so data that is numerical already.

In [ ]:
col_fin = ['mock1','mock2']
tar = ['finalGrade']
X = np.array(testData[col_fin])
y = np.array(testData[tar])

# Define how much test and training data we want. You can try changing these later to see how it changes the model and predictions
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, shuffle=True, random_state=0)

# Create linear regression object
obj = sklearn.linear_model.LinearRegression(fit_intercept=True)

# Train the model using the training sets
obj.fit(X_train, y_train)

# Make predictions using the testing set
y_pred = obj.predict(X_test)

X_test_disp = X_test[:,0] # We will need to make a special vector for the feature we want on the x axis, as now X is a matrix matplotlib can't use it for a scatter plot

# Plot outputs
plt.scatter(X_test_disp, y_test,  color='black', label='y_test') # Observed y values
plt.scatter(X_test_disp, y_pred, color='blue', label='y_pred') # predicted y values
plt.xlabel('Mock 1')
plt.ylabel('Final Grade')
plt.legend()
plt.show()

# The mean squared error loss
print('Mean squared error loss: {:.4f}'.format(sklearn.metrics.mean_squared_error(y_test, y_pred)))
# The R2 score: 1 is perfect prediction
print('R2 score: {:.4f}'.format(sklearn.metrics.r2_score(y_test, y_pred)))

Okay, so around 83% $R^2$ score is a pretty strong baseline, but the categorical columns may be the key to explaining the remaining variation.

## 1.2 Label encoding


Given the original dataset, it is clear we have many categorical features. All these need to be encoded. The [LabelEncoder](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.LabelEncoder.html) class is used to transform the categorical or string values to numerical ones (between 0 and n_classes-1).

In this case we will see what an impact the school has on peoples final grades, as there are 4 schools we should expect this column to be filled with values from '0-3'.

In [ ]:
testData_school = testData.copy() # We will be making copies of the dataset as we will be making changes to the columns as we go along

# create an object of the LabelEncoder class
lblEncoder_X = LabelEncoder()
# apply LblEncoder object to our categorical variables (columns - 'school') using the fit_transform method. This returns the column encoded.
testData_school['school'] = lblEncoder_X.fit_transform(testData_school['school'])

print(testData_school) # See that the 'school' column is now filled with integer values matching the 4 schools

Now let's try including this new numerical feature in our model.

In [ ]:
tar='finalGrade'
col_fin=['mock1','mock2','school']

X = np.array(testData_school[col_fin])
y = np.array(testData_school[tar])

# Define how much test and training data we want. You can try changing these later to see how it changes the model and predictions
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, shuffle=True, random_state=0)

# Create linear regression object
obj = sklearn.linear_model.LinearRegression(fit_intercept=True)

# Train the model using the training sets
obj.fit(X_train, y_train)

# Make predictions using the testing set
y_pred = obj.predict(X_test)

X_test_disp = X_test[:,0] # We will need to make a special vector for the feature we want on the x axis, as now X is a matrix matplotlib can't use it for a scatter plot

# Plot outputs
plt.scatter(X_test_disp, y_test,  color='black', label='y_test') # Observed y values
plt.scatter(X_test_disp, y_pred, color='blue', label='y_pred') # predicted y values
plt.xlabel('Mock 1')
plt.ylabel('Final Grade')
plt.legend()
plt.show()

# The mean squared error loss
print('Mean squared error loss: {:.4f}'.format(sklearn.metrics.mean_squared_error(y_test, y_pred)))
# The R2 score: 1 is perfect prediction
print('R2 score: {:.4f}'.format(sklearn.metrics.r2_score(y_test, y_pred)))

The score has improved, but not by much in this case - but there are still lots of other categorical features we haven't used yet. Let's try looking at the impact of the study time - surely that will have an impact on how well they perform on the test!

In [ ]:
testData_studyTime = testData.copy() # We will be making copies of the dataset as we will be making changes to the columns as we go along

# create an object of the LabelEncoder class
lblEncoder_X = LabelEncoder()
# apply LblEncoder object to our categorical variables (columns - 'school') using the fit_transform method. This returns the column encoded.
testData_studyTime['studyTime_enc'] = lblEncoder_X.fit_transform(testData_studyTime['studyTime'])

display(testData_studyTime)

tar='finalGrade'

col_fin=['mock1','mock2','studyTime_enc']


X = np.array(testData_studyTime[col_fin])
y = np.array(testData_studyTime[tar])

# Define how much test and training data we want. You can try changing these later to see how it changes the model and predictions
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, shuffle=True, random_state=0)

# Create linear regression object
obj = sklearn.linear_model.LinearRegression(fit_intercept=True)

# Train the model using the training sets
obj.fit(X_train, y_train)

# Make predictions using the testing set
y_pred = obj.predict(X_test)

X_test_disp = X_test[:,0] # We will need to make a special vector for the feature we want on the x axis, as now X is a matrix matplotlib can't use it for a scatter plot

# Plot outputs
plt.scatter(X_test_disp, y_test,  color='black', label='y_test') # Observed y values
plt.scatter(X_test_disp, y_pred, color='blue', label='y_pred') # predicted y values
plt.xlabel('Mock 1')
plt.ylabel('Final Grade')
plt.legend()
plt.show()

# The mean squared error loss
print('Mean squared error loss: {:.4f}'.format(sklearn.metrics.mean_squared_error(y_test, y_pred)))
# The R2 score: 1 is perfect prediction
print('R2 score: {:.4f}'.format(sklearn.metrics.r2_score(y_test, y_pred)))

Except it turns out that this adds even less accuracy than the 'school', with no significant improvement from just using the mocks.

**Question:** Can you work out what has gone wrong to cause this feature to have no impact on the accuracy?

##### **Answer:**



It's because of the ordinal nature being imposed by the label encoding. Now intuitively you would think this is the case where the label coding is appropriate - surely more hours studying is better than less?
I would agree, however if you interrogate the table displayed above, you will notice that the label encoder hasn't assigned values that increment correctly with the increase in time (0: 0-60, 1: 121-240, 2: 241-480, 3:61-120), as the label encoder isn't able to discern the order from the string and just applies the number in the order it first sees them in the dataset, which ruins any pattern that might be in the feature and means in fitting the model this feature is effectively ignored.

This highlights a weakness in label encoding, which could be mitigated by manually converting the columns into representative ordinal numbers, or by employing a different encoding strategy - such as one-hot encoding.

##1.3 One Hot Encoding

The final encoding strategy I'll talk about today is [one-hot encoding](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.OneHotEncoder.html#sklearn.preprocessing.OneHotEncoder). For each feature, this expands the columns in the dataset to have one column per category using a 0/1 indicator to mark which category corresponds to each data point.

There is no benefit to using OHE for categorical features with only two categories (as you will just end up with two columns where the 1's and 0's are inverted, which adds nothing to the model). So for binary categories always just use label encoding.

Let's try using the `studyTime` feature again to see how different a solution we get when comparing to the label encoding.

If you are going to be using OHE on a larger scale for your data, I would advise setting up a pipeline to streamline things, as in this example I have just manually added all of the encoded columns to our dataframe.

In [ ]:
# create dataset copy for testing purposes
testDataOHE = testData.copy()

# create OneHotEncoder object
oneHotEncoder = OneHotEncoder()
# and fit the OneHotEncoder object to feature studyTime
onehot_enc = oneHotEncoder.fit(testDataOHE[['studyTime']])
print('The categories are: ', onehot_enc.categories_)

# dum is an array of shape (391,2) containing the one-hot encoding of the feature Gender of the dataframe train_dataset_no_nans
# we make a temporary object to be able to manipulate the extra number of columns
OHT = onehot_enc.transform(testDataOHE[['studyTime']]).toarray()

# we add to the encoded columns to our dataset
testDataOHE['studyTime_0'] = OHT[:,0]
testDataOHE['studyTime_1'] = OHT[:,1]
testDataOHE['studyTime_2'] = OHT[:,2]
testDataOHE['studyTime_3'] = OHT[:,3]

# chack that the new columns correspond to the correct time range (note, they may not be split in order, you can see the order they were split in the print statement above)
display(testDataOHE)

tar='finalGrade'

col_fin=['mock1','mock2','studyTime_0','studyTime_1','studyTime_2','studyTime_3']

X = np.array(testDataOHE[col_fin])
y = np.array(testDataOHE[tar])

# Define how much test and training data we want. You can try changing these later to see how it changes the model and predictions
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, shuffle=True, random_state=0)

# Create linear regression object
obj = sklearn.linear_model.LinearRegression(fit_intercept=True)

# Train the model using the training sets
obj.fit(X_train, y_train)

# Make predictions using the testing set
y_pred = obj.predict(X_test)

X_test_disp = X_test[:,0] # We will need to make a special vector for the feature we want on the x axis, as now X is a matrix matplotlib can't use it for a scatter plot

# Plot outputs
plt.scatter(X_test_disp, y_test,  color='black', label='y_test') # Observed y values
plt.scatter(X_test_disp, y_pred, color='blue', label='y_pred') # predicted y values
plt.xlabel('Mock 1')
plt.ylabel('Final Grade')
plt.legend()
plt.show()

# The mean squared error loss
print('Mean squared error loss: {:.4f}'.format(sklearn.metrics.mean_squared_error(y_test, y_pred)))
# The R2 score: 1 is perfect prediction
print('R2 score: {:.4f}'.format(sklearn.metrics.r2_score(y_test, y_pred)))

Okay, so in this case our one-hot encoded features slightly outperform taking the range average, but it is basically the same accuracy wise.

As is often the case in ML, which method will give you the best results when dealing with categorical data will vary based on your dataset, so it is always worth experimenting with different encoding methods for your categorical data when testing your initial models.

##1.4 Exercise 1

So far we have only looked at two of our categorical features.
Using all of the data, see how accurate a prediction of the final grade you can obtain (I was able to get around 93% $R^2$, but maybe you can do better!)

Some things to try:

*   See how performance changes with different encoding methods for each categorical feature (Label Encoding, One-hot encoding, taking the range average).
*   You can use different encoding methods for different features if appropriate.



In [ ]:
##############################################################
# Your code here
##############################################################

#2. Data preprocessing and the full ML pipline

## 2.1 Data Pre-processing
As described in the lecture, feature scaling has multiple benefits for our ML implementation, including:
- Evening out the sensitivity of the different parameters to changes
- Making the model more interpretable
- Improving regularisation behaviour
- Improving optimisation behaviour (explained in later weeks)

The two main methods we are currently considering are Standardisation and Normalisation (MinMax Scaling), both of which are easy to apply using sklearn.

**Standardization**

By calculating the mean and variance for our data columns we can scale each feature so that it has a mean of 0 and a variance of 1.
$$x_s = \frac{x-\mu_x}{\sigma_x}$$
We can standardize the data by the `sklearn.preprocessing.StandardScaler` instance.
- Initialization: `StandardScaler` initializer.
- Calculating the mean and variance (standard deviation) by using `fit` with our object and data.
  - Input: the `np.array` instance of which to calculate the mean and variance.
- Apply the standardization using `transform`.

In [ ]:
scaler = StandardScaler()
# calculate the mean and variance for each feature and store to attributes
scaler.fit(X_train[:,0:2]) # We only want to perform feature scaling on the numeric (so not categorical) features
print(f'Raw mean = \n{scaler.mean_}, \nRaw var = \n{scaler.var_}') # print the calculated mean and variance for each attribute
X_train_stded = scaler.transform(X_train[:,0:2]) # standardize X_train
# verify that X_train_stded has mean 0 (mean isn't quite 0 due to numerical error, but is a miniscule value) and variance 1.
print(f'Standardised mean = \n{np.mean(X_train_stded, axis=0)}, \nStandardised var = \n{np.var(X_train_stded, axis=0)}')

# Remember to rejoin our scaled features with our categorical features in our final feature matrix
X_train_stded = np.c_[X_train_stded, X_train[:,2:]]
print(X_train_stded)

**Normalisation (MinMax)**

By calculating the maximum and minimum values for our data columns we can scale each feature so that it takes a value between 0 and 1.
$$x_s = \frac{x-x_{min}}{x_{max}-x_{min}}$$
We can standardize the data by the `sklearn.preprocessing.MinMaxScaler` instance.
- Initialization: `MinMaxScaler` initializer.
- Calculating the min and max of the feature by using `fit` with our object and data.
  - Input: the `np.array` instance of which to calculate the mean and variance.
- Apply the standardization using `transform`.

In [ ]:
scaler = MinMaxScaler()
# calculate the mean and variance for each feature and store to attributes
scaler.fit(X_train[:,0:2]) # We only want to perform feature scaling on the numeric (so not categorical) features
print(f'Raw min = \n{np.min(X_train[:,0:2], axis=0)}, \nRaw max = \n{np.max(X_train[:,0:2], axis=0)}') # print the calculated mina and max for each attribute
X_train_stded = scaler.transform(X_train[:,0:2]) # standardize X_train
# verify that X_train_stded has min 0 and max 1.
print(f'Normalised Min = \n{np.min(X_train_stded, axis=0)}, \nNormalised Max = \n{np.max(X_train_stded, axis=0)}')

# Remember to rejoin our scaled features with our categorical features in our final feature matrix
X_train_stded = np.c_[X_train_stded, X_train[:,2:]]
print(X_train_stded)

For most of our case we will apply standardisation, which is generally a better choice for unbounded data like we are using. However MinMax scaling is a valid approach that can be useful when we have a bounded numerical column (for example if we know all values are in a range of 0-100). It is possible that you may want to perform different feature scaling operations on different features depending on the type of data being stored.

Note: Remember that we should generally not perform feature scaling on our categorical features. Though this can cause practical difficulties when implementing polynomial interactions between categorical and numerical columns or big data implementation with many columns of different data types.

##2.2 Data Splitting

So far in our models we have been splitting our data into two sections - training data and test data. However, until this week I have not really explained the rationale behind this in detail, and in fact splitting our data into three groups is generally preferable for a robust ML pipeline.

To begin with, let us load in our California Housing data (staying univariate for now to keep things simple) and convert it to arrays as we usually do:

In [ ]:
# Load the house price dataset
X_pd, y_pd = sklearn.datasets.fetch_california_housing(return_X_y=True, as_frame=True)
Xy_df = pd.concat([X_pd, y_pd], axis=1)

# Use only one feature
col = 'MedInc'
Xy_df = Xy_df[['MedInc', 'MedHouseVal']]

# show the data
display(Xy_df)

# Convert the data to `np.array`
X_raw = np.array(X_pd[['MedInc']])
y = np.array(y_pd)


As we have been exploring over the last few weeks, when optimising parameters, selecting a model, and tuning hyperparameters, **overfitting** happens.
Hence, we need a set of data for optimising our model parameters (training) and an additional set (validation) where we compare the performance of multiple models and hyperparameters to help us decide what our optimal Machine Learning model is for a given dataset. However, we still need to perform a final evaluation of this model using a final set of data (testing) that hasn't been used in either of these prior steps. It may seem counter-intuitive, but even though the validation dataset is not used to train the model parameters directly, we can still end up overfitting the validation data to a degree as we pick the model and hyperparameters which perfoms the best in the validation stage.

In brief - we split the feature/target pairs into the following three datasets:
- **Training data** for optimising parameters.
- **Validation data** for selecting a model and tuning hyperparameters.
- **Test data** for evaluating the true performance of your best model(s)

The exact proportion of training:validation:testing data does not follow exact rules, and may depend on how much data you have available (if you do not have much data, you may not want to use too much test and validate as you may be missing out on information from the features). I would say that as a rule of thumb anywhere between 60%:20%:20% and 80%:10%:10% is an easily justifiable split (ideally keeping the validation and test datasets both using the same proportion).

Strictly speaking, once you split the test data from others, we can adopt a more complicated way to optimise parameters and select a model and tune hyperparameters. The above splitting strategy is one of the simplest way, but widely used. Another widely used example is [**cross validation**](https://scikit-learn.org/stable/modules/cross_validation.html#cross-validation), where we split the non-test data multiple times. There is more detail on this in Appendix 1, and while not required to do well in your coursework it is a powerful and widely used approach to consistent data splitting results that you may want to consider exploring.

From now on we will split the data using two rounds of the `sklearn.model_selection.train_test_split` function. A reminder, we must ensure we use the correct data shuffle setting.
- For non time series data, usually, **we should shuffle** the data by specifying **`shuffle=True`** to avoid the bias of the data by data index.
- For time series data, we **must NOT shuffle** the data to chronologically split the data into training/validation/test data. Specifically, the training data must be earlier data than the other two, and the validation data must be earlier than the test data. In this way, we can simulate the real application, where we handle the latest data.


In [ ]:
# Split the data into non-test/test data
# While we have 20640 pairs of a feature and target, we use 20% only for the testing, not so we hold back the other 80% for training and validation
# `shuffle=True` for non-time series case. You should set `shuffle=False` for time series data to avoid future data being contaminated in the training data.
X_non_test_raw, X_test_raw, y_non_test, y_test = train_test_split(X_raw, y, test_size=0.20, shuffle=True, random_state=0)

# Split the non-test data into non-test/test data
# We use 25% of the remaining non-test data only for the validation set, leaving the rest for training
# In the end, the splitting ratio will be 6:2:2 for the training, validation, and test data.
# `shuffle=True` for non-time series case. You should set `shuffle=False` for time series data to avoid future data being contaminated in the training data.
X_train_raw, X_valid_raw, y_train, y_valid = train_test_split(X_non_test_raw, y_non_test, test_size=0.25, shuffle=True, random_state=0)

Preprocessing reminder (e.g. polynomial features and standardisation/normalisation):

Validation data and test data are both used to simulate new data we are using in the prediction stage. Hence, similar to what we learnt in previous weeks, machine learning prediction models require the features in the training data, validation data, test data (the data on which we predict) to all be preprocessed **the same way**.

For example, we must use the mean and standard deviation **of the training data** for standardizing **both the training data and the new data**.
If we fit a new preprocessing object on our new data then we would obtain different values for the mean an standard deviation such that after preprocessing the same value in the training data and new data would become different in both datasets after preprocessing, confusing our ML model and almost certainly leading to useless predictions.

Hence, we fit the preprocessor to **the training data only**, and use this to transform **both the training data and the new data** (where by *new* data we mean our validation data, test data and any truly *new* data encountered when deploying our ML solution in the real world).

## 2.3 Full ML pipline

I have discussed this in the lecture, but below I would just like to repeat the full Machine Learning pipeline we should be following when fitting any supervised learning models for your easy reference (remember that this all also applies to classification models and even the Neural Network models we will be using in later weeks).

Note: Ensure you understand the distinction between a `fit` command using the preprocessor (e.g., `sklearn.preprocessing.StandardScaler`) vs your predictive model (e.g., `sklearn.linear_model.LinearRegression`).

General pipeline:
- Data loading
  - Ensure you perform and Exploratory Data Analysis (EDA) and clean your data before going any further when dealing with real data.
  - Convert your data into arrays that your ML model can interface with.
  - Split your data into the Train:Validation:Test sets.
- Initialization
  - Initialize the preprocessor(s) and the machine learning model objects.
- Model Selection
  - Identify the different solutions you want to try: this could involve different model types (e.g. Linear Regression, Lasso, Ridge, Decision tree, Neural Network, etc), preprocessing methods, feature selection/engineering and hyperparameter selection.
  - Training (do this for **all** models and hyperparameters)
    - **Fit the preprocessor(s)** to the raw training feature matrix.
    - **Transform (Preprocess)** the raw training feature matrix into the preprocessed training feature matrix.
    - **Fit (Train) the prediction model** using the data pair of the preprocessed training feature matrix and target column vector.
    - Recording the performance of the prediction on training data is often also helpful for identifying overfitting.

  - Validation (for **all** models and hyperparameters)
    - **Transform (Preprocess)** the raw validation feature matrix into the preprocessed validation feature matrix using **the preprocessor(s)** (**Do NOT fit any preprocessor to the validation feature matrix**).
    - **Predict** using the preprocessed validation feature matrix to get a predicted target column.
    - **Evaluate** the prediction performance on the validation data by a loss and/or score function.

  - Continue training and validation until you have a model (or selection of models) you feel are potentially good enough to offer a robust solution to the provided prediction problem.

- Test (for the model and hyperparameters selected by the validation process)
  - **Transform (Preprocess)** the raw test feature matrix into the preprocessed test feature matrix by **the preprocessor(s)** (**Do NOT fit any preprocessor to the test feature matrix**).
  - **Predict** on the preprocessed test feature matrix to get a predicted target column.
  - **Evaluate** the prediction performance on the test data by a loss and/or score function.

If after the test step your model is not providing good enough solutions on the test data, then you will have to go back to rethink how you can create a more robust solution less prone to overfitting.


#3. Hyperparameter selection

Choosing appropriate hyperparameters is one of the reasons we need to include the validation step.

A hyperparameter are parameters that dictates how the model behaves, but are not directly related to any of our features in the data. The main hyperparameter we have encountered so far is the degree of our polynomial regression, but the regularisation weight I have talked about in the lecture (and we will see in the next section) is another example. When I talk to you about Neural Networks in a few weeks we will have even more of these - such as the learning rate, number of epochs and even the number of hidden layers.

Systemically selecting hyperparmaters is very important to obtaining a good ML solution, and while we could always just try varying these by hand this is slow and you need to make sure you keep detailed records of the results. It is far more robust to set up a pipeline to test a range of values for us and provide us with the setup that gives us the lowest Mean Squared Error.

Experimentation is key to obtaining the best results, so ensure you demonstrate you have setup a framework for selecting the best hyperparamters whenever you are making a model recommendation.

##3.1 Hyperparameter Selection Example: Polynomial Regression.

Reminder from last week:

Using [PolynomialFeatures](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.PolynomialFeatures.html?highlight=polynomial#sklearn.preprocessing.PolynomialFeatures) is another preprocessing step, and is very similar to the feature scaling in implementation.

- Initialize the polynomial features object.
  - Method: `PolynomialFeatures(degree)`
  - Input parameter: Maximum polynomial degree you want for your model (this is really a hyperparameter we need to choose carefully, but we will cover how to best pick this degree next week).
- Output: Creates a new feature matrix with polynomial features.
  - Method: `poly.fit_transform`

In this section I will show you how we can select the best degree of polynomial features for the California housing data we loaded in and split above.

Note: The order of the preprocessing steps matters and will change the results - generally it is better to create the polynomial features first then apply standardisation, especially when we have multiple features (see Rule #2 at [this link](https://samchaaa.medium.com/preprocessing-why-you-should-generate-polynomial-features-first-before-standardizing-892b4326a91d)).

First, let's just fit a basic model of a fixed degree to remind you of the full process (make sure you read the comments to understand what we are doing and how this fits into our ML pipeline):

In [ ]:
################
# Initialisation
################
model = sklearn.linear_model.LinearRegression() # Create our prediction model object
poly = PolynomialFeatures(degree=3) # Create our Polynomial Features object, here taking a fixed degree of 3
scaler = StandardScaler() # Here is our standardisation object

##########
# Training
##########
# Create Polynomial features BEFORE standardisation
poly.fit(X_train_raw) # Fit polynomial features on our raw training data
X_train_poly = poly.transform(X_train_raw) # transform our raw data using the poly object

scaler.fit(X_train_poly) # Fit the standardisation object on the polynomial feature matrix
X_train = scaler.transform(X_train_poly) # Standardise the feature matrix to get our 'final' training data feature matrix

model.fit(X_train, y_train) # fit our linear regression model - 'train' the model

# Evaluate performance of the training data
# This does NOT reflect the true performance of the model, but keeping track of the training data performance is helpful for checking how much the model is suffering from overfitting.
y_pred_train = model.predict(X_train) # Use our fitted linear regression model to make a prediction
mse_train = mean_squared_error(y_train, y_pred_train) # Calculate the Mean Squared Error for our training data predictions
print('MSE on training data:', mse_train)

############
# Validation
############

# Preprocess validation dataset using preprocessor objects for on the training data
# Do NOT refit the preprocessors
# Create Polynomial features BEFORE standardisation
X_valid_poly = poly.transform(X_valid_raw) # Add polynomial features to the validation data
X_valid = scaler.transform(X_valid_poly) # Standardise the validation data

# We select the best hyperparameters based on the performance of our model on the validation dataset
y_pred_valid = model.predict(X_valid) # Use our fitted linear regression model to make a prediction based on the validation dataset
mse_valid = mean_squared_error(y_valid, y_pred_valid)  # Calculate the Mean Squared Error for our validation data predictions
print('MSE on validation data:', mse_valid)

# Let's just plot our outputs to see how our model is looking
plt.scatter(X_valid_raw, y_valid,  color='black', label='y_true') # Observed y values
plt.scatter(X_valid_raw, y_pred_valid, color='blue', label='y_pred') # predicted y values
plt.xlabel('MedInc')
plt.ylabel('house price')
plt.legend()
plt.show()


Okay, so from this we've seen that a polynomial degree of 3 behaves similarly for both our training and validation data - but is this the best degrees to pick?

Rather than varying this hyperparameter manually, we want to set up a framework to automate this for us.

In the following we run a `for` loop incrementing the hyperparameter variable `degree`, applying polynomial regression and storing the MSE for the training and validation data in an array.

In [ ]:
max_degree = 30 #Define the max degree to test
# Initialise the MSE arrays, filling them with NaN's
mse_train_array = np.full([max_degree + 1], np.nan)
mse_valid_array = np.full([max_degree + 1], np.nan)
degrees = range(1, max_degree+1) # create list of degree values being iterated through

for degree in degrees:

  # Initialise
  model = sklearn.linear_model.LinearRegression()
  poly = PolynomialFeatures(degree=degree)
  scaler = StandardScaler()
  # Train
  poly.fit(X_train_raw)
  X_train_poly = poly.transform(X_train_raw)
  scaler.fit(X_train_poly)
  X_train = scaler.transform(X_train_poly)
  model.fit(X_train, y_train)
  y_pred_train = model.predict(X_train)
  mse_train = mean_squared_error(y_train, y_pred_train)

  # Validate
  X_valid_poly = poly.transform(X_valid_raw)
  X_valid = scaler.transform(X_valid_poly)
  y_pred_valid = model.predict(X_valid)
  mse_valid = mean_squared_error(y_valid, y_pred_valid)

  # Store MSE for this degree value
  mse_train_array[degree] = mse_train
  mse_valid_array[degree] = mse_valid

Run the cell below to visualise the learning curves from fitting all of these models. You can see that the MSE on the training data monotonously decreases as we use higher degree polynomials (i.e. when using more complex models). As we saw last week, the more complex a model is, the better it can fit training data.

However, the MSE on the validation data starts to increase around `degree=9`  onwards. This shows the overfitting effect. At the beginning of the plot where we have a small `degree` and both the training MSE and validation MSE are high (around `degree = 0-6`) we are seeing underfitting.

We now have the data we need to choose the best degree for our data, as we want to use whichever degree minimises the **validation** MSE, in this case taking `degree = 8`.

In [ ]:
plt.plot(degrees, mse_train_array[1:], label='MSE on training dataset')
plt.plot(degrees, mse_valid_array[1:], label='MSE on validation dataset')
plt.xlabel('Degree of polynomial features')
plt.ylabel('Mean squared error')
plt.title('Degree of polynomial features and mean squared error')
plt.legend()
plt.show()

best_degree = np.nanargmin(mse_valid_array) # Finds the smallest VALIDATION MSE in the array (ignoring any NaN values).
print('The best degree of polynomials:', best_degree)

Finally, now we have selected the best hyperparameter (degree) on the validation data we now think we have our best model for the data. However, it is possible that the model with the selected hyperparameter is actually overfitting the validation data. To check the true performance of our ML model, we must measure the MSE on the test data.

In [ ]:
# Initialise the test instances
model = sklearn.linear_model.LinearRegression()
poly = PolynomialFeatures(degree=best_degree) # use the degree we selected above
scaler = StandardScaler()

# Remember we have to fit our preprocessors and model on the training data again, as we were using many other degrees in our parametric test above
poly.fit(X_train_raw)
X_train_poly = poly.transform(X_train_raw)
scaler.fit(X_train_poly)
X_train = scaler.transform(X_train_poly)
model.fit(X_train, y_train)

mse_train = mean_squared_error(y_train, model.predict(X_train))
print('MSE on the train data:', mse_train) # Get our MSE for the training Data for comparison


X_valid_poly = poly.transform(X_valid_raw)
X_valid = scaler.transform(X_valid_poly)
mse_valid = mean_squared_error(y_valid, model.predict(X_valid))
print('MSE on the valid data:', mse_valid) # Get our MSE for the validation Data for comparison

# Now we have fit our preprocessors using the correct degree, we can process our test data and make a prdiction
X_test_poly = poly.transform(X_test_raw)
X_test = scaler.transform(X_test_poly)
y_pred_test = model.predict(X_test)
mse_test = mean_squared_error(y_test, y_pred_test)
print('MSE on the test data:', mse_test) # check our testing MSE, is it signficantly different from our validation MSE?

# Let's just plot our outputs to see how our model is looking
plt.scatter(X_test_raw, y_test,  color='black', label='y_true') # Observed y values
plt.scatter(X_test_raw, y_pred_test, color='blue', label='y_pred') # predicted y values
plt.xlabel('MedInc')
plt.ylabel('house price')
plt.legend()
plt.show()


# Finally, let's also get the R2 score for our final model
print('R2 score: {:.4f}'.format(sklearn.metrics.r2_score(y_test, y_pred_test)))

So, for this case we can be confident we are not overfitting the validation dataset, with a very similar MSE value to what we observed for `degree=8` during validation.

##3.2 Exercise 2

Use the pipeline outlined above for `degree` selection to find the best polynomial degree for multivariable model for the California housing dataset to see how accurate a final model you can obtain. What do you notice as you include more features and higher polynomial degrees?

Note:
*   Try to use the automated degree selection from above to avoid picking degree values manually like we did last week!
*   Experiment with adding/removing features one at a time by hand, how does this change the result? (we will address removing this chore in the next section). What about changing the amount of training data?
*   Be careful if you are using the full dataset with many features - google Colab may take a long time fitting models with large numbers of polynomial features, if your code is taking too long to run consider lowering the `max_degree` or lowering the size of the sample/number of features being used.

In [ ]:
##############################################################
# Your code here
##############################################################

#4. Regularization
Instead of manually selecting a model from numerous candidates, regularization can be used to help control the model complexity by lowering the parameter values automatically to weigh against unhelpful parameters. While this process isn't perfect, and can still lead to overfitting the validation data, it does present an easy way of including large numbers of features in your model without painstakingly adding and removing individual features manually.

The strength of the regularization can be regarded as a hyperparameter. Hence, we apply the same pipeline as we did for polynomial degree selection.

##4.1 Example: Ridge regression (linear regression with $\ell^2$ regularization)
Linear regression with the $\ell^2$ regularization term is called Ridge regression. This is just one basic form of a model utilising regularisation, essentially following the same process as our other linear models, but with a differently defined cost function being minimised.

The cost function of the ridge regression is given as follows:
$$
J_{\alpha} (\boldsymbol{\theta}) = L (\boldsymbol{\theta}) + R_{\alpha} (\boldsymbol{\theta}),
$$
where
$$
L (\boldsymbol{\theta})
=
\frac{1}{m} \sum_{i=0}^{m-1} (y^{(i)} - {\boldsymbol{x}^{(i)}}^{\top} \boldsymbol{\theta})^{2},
$$
and
$$
R_{\alpha} (\boldsymbol{\theta})
=
\frac{\alpha}{m} \sum_{j=1}^{n-1} (\theta_{j})^2.
$$

Here,
- $L$ is the loss function that gives the mean squared error,
- $R_{\alpha}$ is the regularization function that gives a $\ell^2$ regularization term,
- $\alpha > 0$ is the regularization weight.

  - If $\alpha$ is large, the regularization is strong, and overfitting is strongly avoided, but may cause underfitting.
  - If $\alpha$ is small, the regularization is weak. In this case the model is close to the original model, so it will not prevent any overfitting.

We can implement the ridge regression model by a `sklearn.linear_model.RidgeRegression` instance.
The regularization weight is specified by the parameter `alpha`, which corresponds to $\alpha$, in the initializer of the `RidgeRegression` class.

### 4.1.1 Ridge regression with a fixed $\alpha$

First, let's just fit a basic Ridge regression instance with a regularisation weight $\alpha$ we have chosen. Let's use a polynomial degree of 30 for this case. This follows the same process as earlier models, but now we are defining out `Ridge` model and `alpha` values as well.

In [ ]:
# Load the house price dataset
X_pd, y_pd = sklearn.datasets.fetch_california_housing(return_X_y=True, as_frame=True)

# Convert the data to `np.array`
X_raw = np.array(X_pd[['MedInc']])
y = np.array(y_pd)

# Split the data
X_non_test_raw, X_test_raw, y_non_test, y_test = train_test_split(X_raw, y, test_size=0.20, shuffle=True, random_state=0)
X_train_raw, X_valid_raw, y_train, y_valid = train_test_split(X_non_test_raw, y_non_test, test_size=0.25, shuffle=True, random_state=0)

# Initialise

# Hyperparameters
alpha = 1
degree = 10

# Initialise Model and preprocessors
model = Ridge(alpha=alpha)
poly = PolynomialFeatures(degree=degree)
scaler = StandardScaler()

# Train
poly.fit(X_train_raw)
X_train_poly = poly.transform(X_train_raw)
scaler.fit(X_train_poly)
X_train = scaler.transform(X_train_poly)
model.fit(X_train, y_train) # note, we fit our ridge model exactly the same as our linear models
y_pred_train = model.predict(X_train)
mse_train = mean_squared_error(y_train, y_pred_train)

# Validate
X_valid_poly = poly.transform(X_valid_raw)
X_valid = scaler.transform(X_valid_poly)
y_pred_valid = model.predict(X_valid)
mse_valid = mean_squared_error(y_valid, y_pred_valid)

print('MSE on training data:', mse_train)
print('MSE on validation data:', mse_valid)

# Preprocess test data
X_test_poly = poly.transform(X_test_raw)
X_test = scaler.transform(X_test_poly)

# Predict test data
y_pred_test = model.predict(X_test)
mse_test = mean_squared_error(y_test, y_pred_test)

print('MSE on the test data:', mse_test)

# Plot outputs
plt.scatter(X_test_raw[:,0], y_test,  color='black', label='y_true') # Observed y values
plt.scatter(X_test_raw[:,0], y_pred_test, color='blue', label='y_pred') # predicted y values
plt.xlabel('MedInc')
plt.ylabel('house price')
plt.legend()
plt.show()

# The R2 score: 1 is perfect prediction
print('R2 score: {:.4f}'.format(sklearn.metrics.r2_score(y_test, y_pred_test)))


This seems to be giving us a decent fit to the data already without overfitting, but this is just from arbitrarily picking a regularisation rate. We can actually treat this as a hyperparameter, which means we can implement the same framework used above for polynomial degree to pick the best value for $\alpha$.

###4.1.2 Choosing best $\alpha$ for ridge regression

This is a very similar pipeline to that done above for polynomial degree.
However, unlike the from the previous polynomial case, the preprocessing is the same regardless of the regularization weight (our feature matrix never changes regardless of the regularisation). Hence, we preprocess the training and validation data before the `for` loop. You can focus on the training and evaluating MSE in the `for` loop.

Note: When doing regularisation, we don't need to select the best polynomial degree per se - the regularisation term will in effect account for that to some extent by weighting against unhelpful parameters. However, this does mean that we may have many features in our dataset that are useless, yet are still taking up space/slowing down the fitting processes. I would recommend starting with a relatively small degree and try increasing it if the fitting isn't taking too long.

In [ ]:
#####################
# Load and split data
#####################

# Load the house price dataset
X_pd, y_pd = sklearn.datasets.fetch_california_housing(return_X_y=True, as_frame=True)

# Convert the data to `np.array`
X_raw = np.array(X_pd[['MedInc']])
y = np.array(y_pd)

# Split the data
X_non_test_raw, X_test_raw, y_non_test, y_test = train_test_split(X_raw, y, test_size=0.20, shuffle=True, random_state=0)
X_train_raw, X_valid_raw, y_train, y_valid = train_test_split(X_non_test_raw, y_non_test, test_size=0.25, shuffle=True, random_state=0)

###################################
# Initialise Model and preprocessors
###################################

degree = 10 # Choose your degree. Regularisation will eliminate any values that hurt our model, but do beware about going to big, or python may not be able to cope with the number of features.
alpha_indices = np.arange(20) # Get a list of 1-20
alphas = 10.0 ** (alpha_indices -10) # Define a list of alphas by taking indices from 10 to -10
# Get the MSE arrays
mse_train_array = np.full([len(alphas)], np.nan)
mse_valid_array = np.full([len(alphas)], np.nan)

poly = PolynomialFeatures(degree=degree)
scaler = StandardScaler()

#######################
# Training & validation
#######################

# preprocessing
poly.fit(X_train_raw)
X_train_poly = poly.transform(X_train_raw)
scaler.fit(X_train_poly)
X_train = scaler.transform(X_train_poly)
X_valid_poly = poly.transform(X_valid_raw)
X_valid = scaler.transform(X_valid_poly)

# Loop to test alpha values
for alpha_index, alpha in zip(alpha_indices, alphas):
  # train
  model = Ridge(alpha=alpha) # fits our ridge model with a new regularisation weight each iteration
  model.fit(X_train, y_train)
  y_pred_train = model.predict(X_train)
  mse_train = mean_squared_error(y_train, y_pred_train)

  # Validation
  y_pred_valid = model.predict(X_valid)
  mse_valid = mean_squared_error(y_valid, y_pred_valid)

  # Store MSEs for this iteration
  mse_train_array[alpha_index] = mse_train
  mse_valid_array[alpha_index] = mse_valid
  print(f'alpha: {alpha}, Validation mean squared error: {mse_valid}.')

# Plot our graph of MSE for training and validation
plt.plot(alphas, mse_train_array, label='MSE on training dataset')
plt.plot(alphas, mse_valid_array, label='MSE on validation dataset')
plt.xlabel(r'Regularization weights $\alpha$')
plt.ylabel('Mean squared error')
plt.title(r'Regularization weights $\alpha$ and mean squared error')
plt.xscale('log')
plt.legend()
plt.show()

# Select our best performing alpha (lowest MSE)
best_alpha_index = np.argmin(mse_valid_array)
best_alpha = alphas[best_alpha_index]
print('The best alpha:', best_alpha)

######
# Test
######

#refit our model with the best alpha
model = Ridge(alpha=best_alpha) # fits our ridge model with a new regularisation weight each iteration
model.fit(X_train, y_train)

# Preprocess test data
X_test_poly = poly.transform(X_test_raw)
X_test = scaler.transform(X_test_poly)

# Predict test data
y_pred_test = model.predict(X_test)
mse_test = mean_squared_error(y_test, y_pred_test)

print('MSE on the test data:', mse_test)

# Plot outputs
plt.scatter(X_test_raw[:,0], y_test,  color='black', label='y_true') # Observed y values
plt.scatter(X_test_raw[:,0], y_pred_test, color='blue', label='y_pred') # predicted y values
plt.xlabel('MedInc')
plt.ylabel('house price')
plt.legend()
plt.show()

# The R2 score: 1 is perfect prediction
print('R2 score: {:.4f}'.format(sklearn.metrics.r2_score(y_test, y_pred_test)))

Looking at this result, we can see that the pipeline chose a different value for $\alpha$, providing a slightly better $R^2$ score and lower MSE than we obtained above (in this case the improvement is marginal, but this may not always be the case!).

##4.2 Example: Lasso regression (linear regression with $\ell^1$ regularization)
Linear regression with the $\ell^1$ regularization term is called Lasso regression, compared to the ridge method we used above which utilises $\ell^2$ regularisation. This also works by changing the cost function.

The cost function of the lasso regression is given as follows:
$$
J_{\alpha} (\boldsymbol{\theta}) = L (\boldsymbol{\theta}) + R_{\alpha} (\boldsymbol{\theta}),
$$
where
$$
L (\boldsymbol{\theta})
=
\frac{1}{m} \sum_{i=0}^{m-1} (y^{(i)} - {\boldsymbol{x}^{(i)}}^{\top} \boldsymbol{\theta})^{2},
$$
and
$$
R_{\alpha} (\boldsymbol{\theta})
=
\frac{\alpha}{m} \sum_{j=1}^{n-1} (\theta_{j}).
$$

Here,
- $L$ is the loss function that gives the mean squared error,
- $R_{\alpha}$ is the regularization function that gives a $\ell^1$ regularization term,
- $\alpha > 0$ is the regularization weight.

  - If $\alpha$ is large, the regularization is strong, and overfitting is strongly avoided, but may cause underfitting.
  - If $\alpha$ is small, the regularization is weak. In this case the model is close to the original model, so it will not prevent overfitting.

We can implement the lasso regression model by a `sklearn.linear_model.Lasso` instance.
The regularization weight is specified by the parameter `alpha`, which corresponds to $\alpha$, in the initializer of the `Lasso` class.

Below I have implemented Lasso Regression for a univariate case. The implementation is almost identical, so I have skipped straight to the automatic $\alpha$ selection. Here it seems to be performing similarly to our Ridge regression, but this may not always be the case.

Note: Our sweep through $\alpha$ values may cause some convergence warning to appear for the Lasso, we don't need to worry about this.

In [ ]:
#####################
# Load and split data
#####################

# Load the house price dataset
X_pd, y_pd = sklearn.datasets.fetch_california_housing(return_X_y=True, as_frame=True)

# Convert the data to `np.array`
X_raw = np.array(X_pd[['MedInc']])
y = np.array(y_pd)

# Split the data
X_non_test_raw, X_test_raw, y_non_test, y_test = train_test_split(X_raw, y, test_size=0.20, shuffle=True, random_state=0)
X_train_raw, X_valid_raw, y_train, y_valid = train_test_split(X_non_test_raw, y_non_test, test_size=0.25, shuffle=True, random_state=0)

###################################
# Initialise Model and preprocessors
###################################

degree = 10 # Choose your degree. Regularisation will eliminate any values that hurt our model, but do beware about going to big, or python may not be able to cope with the number of features.
alpha_indices = np.arange(20) # Get a list of 1-20
alphas = 10.0 ** (alpha_indices -10) # Define a list of alphas by taking indices from 10 to -10
# Get the MSE arrays
mse_train_array = np.full([len(alphas)], np.nan)
mse_valid_array = np.full([len(alphas)], np.nan)

poly = PolynomialFeatures(degree=degree)
scaler = StandardScaler()

#######################
# Training & validation
#######################

# preprocessing
poly.fit(X_train_raw)
X_train_poly = poly.transform(X_train_raw)
scaler.fit(X_train_poly)
X_train = scaler.transform(X_train_poly)
X_valid_poly = poly.transform(X_valid_raw)
X_valid = scaler.transform(X_valid_poly)

# Loop to test alpha values
for alpha_index, alpha in zip(alpha_indices, alphas):
  # train
  model = Lasso(alpha=alpha, max_iter=10000) # fits our ridge model with a new regularisation weight each iteration
  model.fit(X_train, y_train)
  y_pred_train = model.predict(X_train)
  mse_train = mean_squared_error(y_train, y_pred_train)

  # Validation
  y_pred_valid = model.predict(X_valid)
  mse_valid = mean_squared_error(y_valid, y_pred_valid)

  # Store MSEs for this iteration
  mse_train_array[alpha_index] = mse_train
  mse_valid_array[alpha_index] = mse_valid
  print(f'alpha: {alpha}, Validation mean squared error: {mse_valid}.')

# Plot our graph of MSE for training and validation
plt.plot(alphas, mse_train_array, label='MSE on training dataset')
plt.plot(alphas, mse_valid_array, label='MSE on validation dataset')
plt.xlabel(r'Regularization weights $\alpha$')
plt.ylabel('Mean squared error')
plt.title(r'Regularization weights $\alpha$ and mean squared error')
plt.xscale('log')
plt.legend()
plt.show()

# Select our best performing alpha (lowest MSE)
best_alpha_index = np.argmin(mse_valid_array)
best_alpha = alphas[best_alpha_index]
print('The best alpha:', best_alpha)

######
# Test
######

#refit our model with the best alpha
model = Lasso(alpha=best_alpha, max_iter=1000, tol=1e-1) # fits our lasso model with a new regularisation weight each iteration
model.fit(X_train, y_train)

# Preprocess test data
X_test_poly = poly.transform(X_test_raw)
X_test = scaler.transform(X_test_poly)

# Predict test data
y_pred_test = model.predict(X_test)
mse_test = mean_squared_error(y_test, y_pred_test)

print('MSE on the test data:', mse_test)

# Plot outputs
plt.scatter(X_test_raw[:,0], y_test,  color='black', label='y_test') # Observed y values
plt.scatter(X_test_raw[:,0], y_pred_test, color='blue', label='y_test') # predicted y values
plt.xlabel('MedInc')
plt.ylabel('house price')
plt.legend()
plt.show()

# The R2 score: 1 is perfect prediction
print('R2 score: {:.4f}'.format(sklearn.metrics.r2_score(y_test, y_pred_test)))

## 4.3 Exercise 3


Try experimenting with the regularised regression models (Lasso and Ridge) to fit the best multivariable model you can on the California housing dataset.

1.   To begin with, try sample size of `1000`, degree `5` and using all of the features, as this small sample should speed up testing.
2.   Extra: Try manually removing some of the original features (so the features we have before fitting our polynomial features object) one at a time. How does this change the speed and the model accuracy?
3.   Extra: Try changing the degree and sample size. How does the solution speed and best regularisation parameter change as the sample both increases and decreases?
4.   Extra: Check the `model.coef_` values for your best model for Lasso and Ridge regression. What do you notice?


Note:
*   We want to use the automated $\alpha$ selection from above, don't pick degree values manually!
*   Above we were looking at small regularisation rates - we might want stronger regularisation in this case, so you may want to explore a different range of $\alpha$ indices.
*   Be careful if you are using the full dataset with many features - google colab may take a long time fitting models with large numbers of polynomial features, if your code is taking too long to run consider lowering the `degree`, lowering the size of the sample or removing the less impactful features (check the `model.coef_` values to identify which of the coefficients are contributing more to the prediction. Because our features are standardised, we can directly compare the magnitude of our coefficients this way).
-  Extra: Consider using [cross validation](https://scikit-learn.org/stable/modules/cross_validation.html#cross-validation) and/or [Grid Search](https://scikit-learn.org/0.17/modules/grid_search.html#grid-search) (see appendices) during your model selection.

In [ ]:
##############################################################
# Your code here
##############################################################

##4.4 Exercise 4

You should now have all the fundamental tools you need to systematically choose the best regression model for a given dataset for a traditional Machine Learning problem. (Though by all means do your own research into other models and methods you can apply to supervised learning problems!)

For this exercise, use the dataset on the compressive strength of concrete(`Concrete_Data.csv`) which I have uploaded to moodle and see how good a regression model you can implement using all the tools we have discussed over the last two weeks. With a little experimentation I was able to get an $R^2$ score of about 85% - can you beat me?

Notes:
- This Concrete dataset is also available at the following link donated by Prof. I-Cheng Yeh: https://archive.ics.uci.edu/ml/datasets/Concrete+Compressive+Strength
- With only 1000 datapoints, you can probably work with the whole dataset, but if you want to explore high polynomial features (say above 10) in a reasonable time you might have to work with a smaller sample and/or remove features to test things in a reasonable time.
-  Extra: Consider using [cross validation](https://scikit-learn.org/stable/modules/cross_validation.html#cross-validation) and/or [Grid Search](https://scikit-learn.org/0.17/modules/grid_search.html#grid-search) (see appendices) during your model selection.
- Extra: You could see how non parametric methods perform (e.g. Decision Trees & K-nearest neighbours)

In [ ]:
##############################################################
# Your code here
##############################################################

##4.5 Exercise 5

If you have completed everything else, you could go back to our example of school test results from **Exercise 1**  (`testData.csv`) with the categorical features. Take the answer you got for this exercise, but now try applying all of the other things we've seen in this week's tutorial to see if we can improve our accuracy! I was able to get nearly 98%, can we do any better?

Some things to try:

*   Perform more robust data splitting with train/validation/test sets.
*   Perform pre-processing on the numerical features
*  See if polynomial features have an impact.
*  You can use Ridge/Lasso regularisation (or even try a non parametric approach like a [decision tree](https://scikit-learn.org/stable/modules/tree.html#tree)).
*  Extra: Consider using cross validation and/or grid searches (see appendices) during your model selection.

In [ ]:
##############################################################
# Your code here
##############################################################

# Appendix 1: Cross Validation

Cross validation is an alternative way to evaluate your models during the model selection process.

Rather than splitting your data into train/validation/test sets, initially you only split it into train/test sets.

Then the training date is split into `k` folds (essentially subsamples of the training data). You then fit your model `k` times, using `k-1` of the folds for training and `1` fold for validation (changing the validation fold each time). You then take the average of the performance metrics (MSE/$R^2$), which gives you a more thorough idea of if your model is robust as by the end of this process it has been tested on all of the training data. If it passes the k-fold cross validation and you are happy with the average performance, you can then fit the model again on the ***entire*** training dataset and evaluate this final model using the test data.

Issues with this is of course it is more computationally expensive as you are fitting `k-1` more models (which isn't a big deal with small datasets, but makes this approach impractical for larger datasets). Also it complicates the process for testing for hyperparameters as we would then have to include the cross validation in that wider loop (often if you want to perform a study with cross validation you would use some sort of pre-made grid search method - see the next appendix).

You are not required to implement Cross validation to do well in your coursework, but you may implement it if you wish as it is a widely used and thorough way to evaluate your model prior to testing.

See the sklearn documentation [here](https://scikit-learn.org/stable/modules/cross_validation.html#cross-validation) for more details.

In [ ]:
from sklearn.model_selection import KFold

# Load the house price dataset
X_pd, y_pd = sklearn.datasets.fetch_california_housing(return_X_y=True, as_frame=True)

# Extract the Median Income feature
X_raw = np.array(X_pd)
y_raw = np.array(y_pd)

# Split the data into training/test data
X_train, X_test, y_train, y_test = train_test_split(X_raw, y_raw, test_size=0.20, shuffle=True, random_state=0)

# Create linear regression object
model = LinearRegression()

# Define K-fold Cross Validation object

# n_splits - how many 'folds' you want to use to evalaute your model.
# 5 folds is pretty standard, I wouldn't advise going much smaller than that, and unless you have a good rationale I would generally not go above 10.
# While generally you want the k-fold splitting to give you folds of a similar size to your test data, this doesn't have to be too closely monitored.
# You want to have a balance between what is computationally sensible with having enough folds to ensure you are happy with the testing thoroughness.

# shuffle - shuffles your data while performing the k-fold splitting. Generally you want to do this as it just helps make the process even more robust.
# random state - give the random seed for the shuffling. This means you can generate the same pseudorandom sample each time to aid comparison.
kf = KFold(n_splits=5, shuffle=True, random_state=0)

# Create empty lists to fill with out MSE and R2 values
mse_list = []
r2_list = []

for train_index, valid_index in kf.split(X_train):
    print('Validation points:', valid_index)
    print('Training points:',train_index)

    # split our full training data into the CV train and validation for this fold
    X_train_cv, X_valid_cv = X_train[train_index], X_train[valid_index]
    y_train_cv, y_valid_cv = y_train[train_index], y_train[valid_index]

    # Train the model
    model.fit(X_train_cv, y_train_cv)

    # Make predictions on the validation set
    y_pred_valid = model.predict(X_valid_cv)

    # Calculate metrics
    mse_valid = mean_squared_error(y_valid_cv, y_pred_valid)
    r2_valid = r2_score(y_valid_cv, y_pred_valid)

    print(mse_valid)
    print(r2_valid)
    print(' ')

    # Store results for this fold
    mse_list.append(mse_valid)
    r2_list.append(r2_valid)

# Calculate average MSE and R2 across folds
avg_mse = np.mean(mse_list)
avg_r2 = np.mean(r2_list)

print('Average MSE on validation data:', avg_mse)
print('Average R2 score on validation data:', avg_r2)

# Now, let's train the model on the full training set and evaluate on the test set
model.fit(X_train, y_train)
y_pred_test = model.predict(X_test)

# Plot outputs
plt.scatter(X_test[:, 0], y_test, color='black', label='y_test')  # Observed y values
plt.scatter(X_test[:, 0], y_pred_test, color='blue', label='y_pred')  # Predicted y values
plt.xlabel('MedInc')
plt.ylabel('House Price')
plt.legend()
plt.show()

# The mean squared error loss on the test set
mse_test = mean_squared_error(y_test, y_pred_test)
print('Mean squared error loss on test data: {:.4f}'.format(mse_test))

# The R2 score on the test set: 1 is perfect prediction
r2_test = r2_score(y_test, y_pred_test)
print('R2 score on test data: {:.4f}'.format(r2_test))


Below is an expanded example where we are using cross validation to help us chose an optimal regularisation strength. While certainly workable, it significantly increases the computational load and does complicate our code further - so depending on how much we want to test a grid search may be a better choice.

In [ ]:
#####################
# Load and split data
#####################

# Load the house price dataset
X_pd, y_pd = sklearn.datasets.fetch_california_housing(return_X_y=True, as_frame=True)

# Convert the data to `np.array`
X_raw = np.array(X_pd[['MedInc']])
y = np.array(y_pd)

# Split the data
X_train_raw, X_test_raw, y_train, y_test = train_test_split(X_raw, y, test_size=0.20, shuffle=True, random_state=0)

###################################
# Initialise Model and preprocessors
###################################

degree = 5 # Choose your degree. Regularisation will eliminate any values that hurt our model, but do beware about going to big, or python may not be able to cope with the number of features.
alpha_indices = np.arange(10) # Get a list of 1-10
alphas = 10.0 ** (alpha_indices -5) # Define a list of alphas by taking indices from 5 to -5
# Get the MSE arrays
mse_train_array = np.full([len(alphas)], np.nan)
mse_valid_array = np.full([len(alphas)], np.nan)

poly = PolynomialFeatures(degree=degree)
scaler = StandardScaler()

#######################
# Training & validation
#######################

# preprocessing
poly.fit(X_train_raw)
X_train_poly = poly.transform(X_train_raw)
scaler.fit(X_train_poly)
X_train = scaler.transform(X_train_poly)

# Loop to test alpha values
for alpha_index, alpha in zip(alpha_indices, alphas):
  # train
  model = Ridge(alpha=alpha) # fits our ridge model with a new regularisation weight each iteration
  mse_list_tr = []
  mse_list_val = []

  for train_index, valid_index in kf.split(X_train):
    # split our full training data into the CV train and validation for this fold
    X_train_cv, X_valid_cv = X_train[train_index], X_train[valid_index]
    y_train_cv, y_valid_cv = y_train[train_index], y_train[valid_index]

    # Train the model
    model.fit(X_train_cv, y_train_cv)

    # Make predictions
    y_pred_train = model.predict(X_train_cv)
    y_pred_valid = model.predict(X_valid_cv)

    # Calculate metrics
    mse_train = mean_squared_error(y_train_cv, y_pred_train)
    mse_valid = mean_squared_error(y_valid_cv, y_pred_valid)

    # Store results for this fold
    mse_list_tr.append(mse_train)
    mse_list_val.append(mse_valid)

    # Calculate average MSE across folds
    avg_mse_val = np.mean(mse_list_tr)
    avg_mse_train = np.mean(mse_list_val)

  # Store MSEs for this iteration
  mse_train_array[alpha_index] = avg_mse_train
  mse_valid_array[alpha_index] = avg_mse_val
  print(f'alpha: {alpha}, Validation average mean squared error: {mse_valid}.')

# Plot our graph of MSE for training and validation
plt.plot(alphas, mse_train_array, label='MSE on training dataset')
plt.plot(alphas, mse_valid_array, label='MSE on validation dataset')
plt.xlabel(r'Regularization weights $\alpha$')
plt.ylabel('Mean squared error')
plt.title(r'Regularization weights $\alpha$ and mean squared error')
plt.xscale('log')
plt.legend()
plt.show()

# Select our best performing alpha (lowest MSE)
best_alpha_index = np.argmin(mse_valid_array)
best_alpha = alphas[best_alpha_index]
print('The best alpha:', best_alpha)

######
# Test
######

#refit our model with the best alpha
model = Ridge(alpha=best_alpha) # fits our ridge model with a new regularisation weight each iteration
model.fit(X_train, y_train)

# Preprocess test data
X_test_poly = poly.transform(X_test_raw)
X_test = scaler.transform(X_test_poly)

# Predict test data
y_pred_test = model.predict(X_test)
mse_test = mean_squared_error(y_test, y_pred_test)

print('MSE on the test data:', mse_test)

# Plot outputs
plt.scatter(X_test_raw[:,0], y_test,  color='black', label='y_true') # Observed y values
plt.scatter(X_test_raw[:,0], y_pred_test, color='blue', label='y_pred') # predicted y values
plt.xlabel('MedInc')
plt.ylabel('house price')
plt.legend()
plt.show()

# The R2 score: 1 is perfect prediction
print('R2 score: {:.4f}'.format(sklearn.metrics.r2_score(y_test, y_pred_test)))

#Appendix 2: Grid Search

Using grid search is an alternative to setting up your own studies (like we have been doing so far). This is a predefined framework in `sklearn` that can be test models using a variety of user defined hyperparameters to find the combination that provides the best results. Of course, this can be a very slow process and many models are being fitted and evaluated - so while grid search does allow you to test a lot of combinations, it is useful if you have the knowledge to provide it with smaller lists of options that you believe are most likely to obtain good results - as python risks timing out and/or running out of memory if you provide too large a list.

You are free to use Grid Searches in your coursework (but this is not required for a top grade), but if you do so some things I want you to bear in mind:
- Grid search does not magically give you the 'best' answer. You as the developer are choosing what hyperparameters to test and what values to use, along with other choices such as preprocessing and data splitting that can have a massive impact on the results.
- It's also impossible to test every permutation of hyperparameter values, so it's always possible a different permutation you never tested will give you a better model.
- You don't have to end your model selection with a single grid search. You can investigate the results and this may give you an idea of different parameters to try testing (either manually now or with further grid searches).
- You don't have to accept the top result of a GridSeach as your final model choice - there are reasons other than the final score function for why you may choose one model over another.

if you use grid search in your coursework I would expect you to clearly show what hyperparameters and hyperparameter values have been chosen to test, with some justifications give for these choices. Further interrogation of the grid search results and potentially even further testing (manual or using more grid searches) before selecting a final model will be looked on favourably.

I'm going to be using `GridSearchCV`, which includes cross validation in it's selection process, but there are [other inbuilt search methods](https://scikit-learn.org/stable/modules/grid_search.html#grid-search) in sklearn you can experiment with.

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline

# Load the house price dataset
X_pd, y_pd = fetch_california_housing(return_X_y=True, as_frame=True)

# Convert the data to `np.array`
X_raw = np.array(X_pd)
y = np.array(y_pd)

# Split the data
X_train_raw, X_test_raw, y_train, y_test = train_test_split(X_raw, y, test_size=0.1, shuffle=True, random_state=0)

# Hyperparameter grid for GridSearchCV
param_grid = {
    'ridge__alpha': [0.01, 0.1, 1, 10],  # Values for Ridge alpha
    'poly__degree': [1, 2, 3, 4, 5],  # Values for PolynomialFeatures degree
}

# Initialize Model and preprocessors
ridge = Ridge()
poly = PolynomialFeatures()  # Apply PolynomialFeatures only to the selected feature
scaler = StandardScaler()

# Create a pipeline with preprocessing and Ridge regression
# Pipelines can be used outside of grid searchs to give you a tidier process
# essentially they roll up multiple preprocesing operations with the model so it only requires 'fitting' once
pipeline = Pipeline([
    ('poly', poly),
    ('scaler', scaler),
    ('ridge', ridge)
])

# Initialize GridSearchCV
# Pipeline - provide teh model (or pipeline) that you want to test teh grid search on
# param_grid - the hyperparameter values you wish to test. Must make sure these
# align with the hyperparameters of the pipeline being fitted or there will be an error.
# cv - cross validation folds to use
# scoring - how to evaluate the different models, many options, but for our regression problem I have chosen R2.
# n_jobs - this allows parallel processing to be use multiple processors to run the search quicker.
# Unless you have a good reason, I would leave as '-1' so all available processors are used.
grid_search = GridSearchCV(pipeline, param_grid, cv=5, scoring='r2', n_jobs=-1)

# Fit the model with GridSearchCV
grid_search.fit(X_train_raw, y_train)

# Get the best hyperparameters
best_params = grid_search.best_params_

# Print the best hyperparameters
print("Best Hyperparameters:", best_params)

# Get the best model
best_model = grid_search.best_estimator_

# Print the best model
print("Best Model:", best_model)

# Train the best model on the full training set
best_model.fit(X_train_raw, y_train)

# Predict test data
y_pred_test = best_model.predict(X_test_raw)
mse_test = mean_squared_error(y_test, y_pred_test)

print('MSE on the test data:', mse_test)

# Plot outputs
plt.scatter(X_test_raw[:, 0], y_test,  color='black', label='y_true')  # Observed y values
plt.scatter(X_test_raw[:, 0], y_pred_test, color='blue', label='y_pred')  # Predicted y values
plt.xlabel('MedInc')
plt.ylabel('house price')
plt.legend()
plt.show()

# The R2 score: 1 is perfect prediction
print('R2 score: {:.4f}'.format(r2_score(y_test, y_pred_test)))


If you are using grid search properly, you should also be looking into all of the results for the entire set of combinations tested. This is good for giving you a better idea of what is and isn't working well (that may be used for further experimentation). You may also look at these results and decide that the 'best' model isn't actually the one you want to use - for example you may have a simpler model that only marginally comes in second place, but will be much more robust when deployed.

One way of doing this is shown below, where all of the results have been put into a sorted table.

In [ ]:
# Get the results as a DataFrame
cv_results = pd.DataFrame(grid_search.cv_results_)

# Display the relevant columns
relevant_columns = ['params', 'mean_test_score', 'std_test_score']
cv_results[relevant_columns].sort_values(by='mean_test_score', ascending=False)


Grid searches can potentially be used with almost any model (it is possible, but more complicated with Neural Networks...). So as an additional example, here is a Decision tree model fitted using grid search.

In [ ]:
from sklearn.tree import DecisionTreeRegressor

# Load the house price dataset
X_pd, y_pd = fetch_california_housing(return_X_y=True, as_frame=True)

# Convert the data to `np.array`
X_raw = np.array(X_pd)
y = np.array(y_pd)

# Split the data
X_train_raw, X_test_raw, y_train, y_test = train_test_split(X_raw, y, test_size=0.1, shuffle=True, random_state=0)

# Hyperparameter grid for GridSearchCV
param_grid = {
    'poly__degree': [1, 2],  # Values for PolynomialFeatures degree
    'tree__max_depth': [None, 5, 10],  # Values for DecisionTreeRegressor max depth
    'tree__min_samples_split': [None, 2, 5],  # Values for DecisionTreeRegressor min_samples_split
}

# Initialize Model and preprocessors
tree = DecisionTreeRegressor()
poly = PolynomialFeatures()  # Apply PolynomialFeatures only to the selected feature
scaler = StandardScaler()

# Create a pipeline with preprocessing and Decision Tree regression
pipeline = Pipeline([
    ('poly', poly),
    ('scaler', scaler),
    ('tree', tree)
])

# Initialize GridSearchCV
grid_search = GridSearchCV(pipeline, param_grid, cv=5, scoring='r2', n_jobs=-1)

# Fit the model with GridSearchCV
grid_search.fit(X_train_raw, y_train)

# Get the best hyperparameters
best_params = grid_search.best_params_

# Print the best hyperparameters
print("Best Hyperparameters:", best_params)

# Get the best model
best_model = grid_search.best_estimator_

# Print the best model
print("Best Model:", best_model)

# Train the best model on the full training set
best_model.fit(X_train_raw, y_train)

# Predict test data
y_pred_test = best_model.predict(X_test_raw)
mse_test = mean_squared_error(y_test, y_pred_test)

print('MSE on the test data:', mse_test)

# Plot outputs
plt.scatter(X_test_raw[:, 0], y_test,  color='black', label='y_true')  # Observed y values
plt.scatter(X_test_raw[:, 0], y_pred_test, color='blue', label='y_pred')  # Predicted y values
plt.xlabel('MedInc')
plt.ylabel('house price')
plt.legend()
plt.show()

# The R2 score: 1 is perfect prediction
print('R2 score: {:.4f}'.format(r2_score(y_test, y_pred_test)))


In [ ]:
# Get the results as a DataFrame
cv_results = pd.DataFrame(grid_search.cv_results_)

# Display the relevant columns
relevant_columns = ['params', 'mean_test_score', 'std_test_score']
cv_results[relevant_columns].sort_values(by='mean_test_score', ascending=False)

#Appendix 3: Missing Data

As mentioned in week 1, on this module we're not having too big a focus on the data preparation side of Data Science / Machine Learning, however it is worth noting at this point how we can deal with missing data using imputation methods, which are another preprocessing method mentioned in the lecture. This provides you with an another option of what to do if your data contains missing values other than dropping those entries (Note: there is no missing data in your coursework, so you don't have to worry about doing this).

These work in a similar way to our other preprocessing functions - you fit them on the training data and transform the training and validation/test data.

The type of imputation you use depends on if you are missing a numeric or categorical data, but all can be accessed using the [SimpleImputer](https://scikit-learn.org/stable/modules/generated/sklearn.impute.SimpleImputer.html) object in sklearn.

Numeric:
- Mean - The average value in the column (generally the default choice for numerical columns)
- Median - The middle value when ordering the column (can be better than the average when your data has outliers or a significant skew)
- Mode - The most commonly occurring value in the column (unusual for numeric)
- Constant - Replaces all missing values with a constant value defined by you (can be useful if you have some domain knowledge telling you how to treat these missing values)

Categorical:
- Mode - The most commonly occurring category in the column (Generally the default choice for a categorical column)
- Constant - Replaces all missing values with a constant category value defined by you (can be useful if you have some domain knowledge telling you how to treat these missing values)

Below I have an example of the school test dataset from exercise 1, but with missing data (`testData_missing.csv` on moodle). Try exploring how different imputation strategies impact the result.

In [ ]:
# Load new data (make sure to manually load into your python environment)
testData = pd.read_csv('testData_missing.csv') # Save it to a pandas dataframe

# Display only rows with missing data
missing_data_rows = testData[testData.isna().any(axis=1)]
display(missing_data_rows)

In [ ]:
from sklearn.impute import SimpleImputer

# Load the data
testData = pd.read_csv('testData_missing.csv')

# Save it to a pandas dataframe
tar = 'finalGrade'

# create an object of the LabelEncoder class
lblEncoder_X = LabelEncoder()

# Impute missing values for numeric columns with mean
numeric_cols = testData.select_dtypes(include=[np.number]).columns

#Below are the differnt approaches you could use for a numeric column - try uncommenting the others to see their impact.
numeric_imputer = SimpleImputer(strategy='mean')
#numeric_imputer = SimpleImputer(strategy='most_frequent')
#numeric_imputer = SimpleImputer(strategy='median')
#numeric_imputer = SimpleImputer(strategy='constant', fill_value = 0) # try messing around with other constant values!

testData[numeric_cols] = numeric_imputer.fit_transform(testData[numeric_cols])

# Impute missing values for categorical columns with mode
categorical_cols = testData.select_dtypes(include=['object']).columns


# Below are the differnt approaches you could use for a categorical column
categorical_imputer = SimpleImputer(strategy='most_frequent')
  # Blanketly applying the constant this way means that all columns will have the same value imputed
  # If you wanted to do anythin other than make a group for missing categories like
  # I have done, you would have to start appling the constant imputation to
  # individual columns so you can manually set what you want done for each feature.
#categorical_imputer = SimpleImputer(strategy='constant', fill_value = 'Missing')

testData[categorical_cols] = categorical_imputer.fit_transform(testData[categorical_cols])

# Encode our categorical columns
testData['studyTime'] = lblEncoder_X.fit_transform(testData['studyTime'])
testData['travelTime'] = lblEncoder_X.fit_transform(testData['travelTime'])
testData['absence'] = lblEncoder_X.fit_transform(testData['absence'])
testData['school'] = lblEncoder_X.fit_transform(testData['school'])

col_fin = ['mock1', 'mock2', 'studyTime', 'travelTime', 'absence', 'school']

X = np.array(testData[col_fin])
y = np.array(testData[tar])

# Define how much test and training data we want. You can try changing these later to see how it changes the model and predictions
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, shuffle=True, random_state=0)

# Create linear regression object
obj = sklearn.linear_model.LinearRegression()

# Train the model using the training sets
obj.fit(X_train, y_train)

# Make predictions using the testing set
y_pred = obj.predict(X_test)

X_test_disp = X_test[:, 0]  # We will need to make a special vector for the feature we want on the x axis, as now X is a matrix matplotlib can't use it for a scatter plot

# Plot outputs
plt.scatter(X_test_disp, y_test, color='black', label='y_test')  # Observed y values
plt.scatter(X_test_disp, y_pred, color='blue', label='y_pred')  # Predicted y values
plt.xlabel('Mock 1')
plt.ylabel('Final Grade')
plt.legend()
plt.show()

# The mean squared error loss
print('Mean squared error loss: {:.4f}'.format(sklearn.metrics.mean_squared_error(y_test, y_pred)))
# The R2 score: 1 is perfect prediction
print('R2 score: {:.4f}'.format(sklearn.metrics.r2_score(y_test, y_pred)))
